In [33]:
import tensorflow as tf

import numpy as np
import glob
from matplotlib import pyplot as plt

In [46]:
model = tf.keras.models.load_model('weights/mobilnet_hd_func.h5', compile=False)

In [ ]:
model.summary()

In [ ]:
tf.saved_model.save(model, 'weights/mobilenet_hd')

# Load saved model

In [38]:
labels = {
    0: 'not hot dog',
    1: 'hot dog'
}

In [44]:
model = tf.saved_model.load('weights/mobilenet_hd/')

In [45]:
infer = model.signatures['serving_default']

In [5]:
images = glob.glob('data/test/**/*.jpg', recursive=True)

In [ ]:
459 ms ± 3.53 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [ ]:
%%timeit
row, column, count = 3, 5, 0

plt.figure(figsize=(15, 10))
for image in np.random.choice(images, 15):
    img = tf.io.decode_jpeg(tf.io.read_file(image))
    img_pre = tf.cast(img, tf.float32)
    img_pre = img_pre / 255.
    img_pre = tf.image.resize(img_pre, [224, 224])
    img_pre = tf.expand_dims(img_pre, axis=0)
#     preds = infer(img_pre)['outputs']
    preds = model.predict(img_pre)
    preds = tf.squeeze(preds, axis=0)
    
    label = labels[preds.numpy().argmax()]
    count += 1
    plt.subplot(row, column, count)
    plt.title(label + ' : ' + str(round(preds.numpy().max() * 100, 2)))
    plt.imshow(tf.cast(img_pre[0] * 255, tf.uint8))
    
plt.tight_layout()